## Data Processing

#### Loading in the Cloud sensor data 

In [1]:
import pandas as pd
import numpy as np
import os

#specifiying directories for the data from the two sites
sodankyla_dir = r"/Users/elliotdable/Documents/UCL/research/fabry_perot_ml/sodankyla_fpi/"
kiruna_dir = r"/Users/elliotdable/Documents/UCL/research/fabry_perot_ml/kiruna_fpi/"

In [5]:
def cloud_sensor_file_loader(directory):
    '''This function loads in all of the cloud sensor csv files in a specfied directory and concatenates them simultaneously.

    Inputs:
        directory: the directory of cloud sensor csv files
        
    Returns:
        dataframe: a concatenated pandas dataframe of all of the csv files containing the cloud data
    '''
    files = os.listdir(directory)

    # Filter only CSV files
    csv_files = [file for file in files if file.startswith('Cloud') and file.endswith('.csv')]

    # Initialize an empty DataFrame to store the concatenated data
    dataframe = pd.DataFrame()

    # Iterate over each CSV file and append it to the end of the concatenated DataFrame
    for file in csv_files:
        filepath = os.path.join(directory, file)
        df = pd.read_csv(filepath, delimiter=',')

        #having to shift the columns and reset index as the csv file wont read in properly
        #this means ulitmately dropping the sensor ID column, which is useless anyway 
        df = df.shift(axis=1)
        df = df.reset_index(drop=True)

        dataframe = pd.concat([dataframe, df])
    
    #dropping sensor ID column
    dataframe = dataframe.drop(['Sensor ID', 'HardwareID'], axis=1)
    
    #converting the unix timestamp to datetime and renaming the columns in camel-case for consistency
    dataframe['UnixTimestamp'] = pd.to_datetime(dataframe['UnixTimestamp'], unit='s')
    dataframe.rename(columns={"UnixTimestamp": "datetime", "Date": "date", "Time": "time", "SensorT":"sensorTemp", "SkyT": "skyTemp", "Clarity": "clarity", "Light":"light", "Rain":"rain"}, inplace=True)

    # dropping date and time columns as we now have a unified datetime column which is more flexible to work with 
    dataframe = dataframe.drop(['date', 'time'], axis=1)
    
    # sort the dataframe by date descedning and reset the idnex for a clean dataset
    dataframe = dataframe.sort_values(by='datetime')
    dataframe = dataframe.reset_index(drop=True)

    return dataframe

In [6]:
sodankyla_cloud_data = cloud_sensor_file_loader(sodankyla_dir)
sodankyla_cloud_data.to_csv(sodankyla_dir + "sodankyla_cloud_data.csv", index=False)
sodankyla_cloud_data

,datetime,sensorTemp,skyTemp,clarity,light,rain
0,2017-02-01 00:00:01,5.4,-45.7,51.1,0.0,0.0
1,2017-02-01 00:01:01,5.4,-45.8,51.2,0.0,-0.1
2,2017-02-01 00:02:02,5.4,-45.4,50.8,0.0,0.0
3,2017-02-01 00:03:03,5.4,-46.3,51.7,0.0,0.0
4,2017-02-01 00:04:03,5.5,-45.2,50.7,0.0,0.0
...,...,...,...,...,...,...
286020,2024-02-08 19:15:05,-3.2,-54.3,51.1,0.0,0.0
286021,2024-02-08 19:16:05,-3.2,-54.7,51.5,0.0,-0.1
286022,2024-02-08 19:17:06,-3.2,-53.2,50.0,0.0,-0.1
286023,2024-02-08 19:18:06,-3.1,-54.8,51.8,0.0,-0.1


In [7]:
kiruna_cloud_data = cloud_sensor_file_loader(kiruna_dir)
kiruna_cloud_data.to_csv(kiruna_dir + "kiruna_cloud_data.csv", index=False)
kiruna_cloud_data

,datetime,sensorTemp,skyTemp,clarity,light,rain
0,2019-02-01 00:00:03,-0.3,-4.4,4.1,0.0,0.4
1,2019-02-01 00:01:03,-0.3,-4.5,4.2,0.0,0.5
2,2019-02-01 00:02:03,-0.3,-4.4,4.1,0.0,0.4
3,2019-02-01 00:03:04,-0.3,-4.6,4.3,0.0,0.4
4,2019-02-01 00:04:04,-0.3,-4.6,4.3,0.0,0.4
...,...,...,...,...,...,...
214193,2024-02-08 19:17:00,-2.3,-10.6,8.2,0.0,0.5
214194,2024-02-08 19:18:00,-2.3,-11.1,8.8,0.0,0.7
214195,2024-02-08 19:19:01,-2.3,-10.6,8.3,0.0,0.5
214196,2024-02-08 19:20:01,-2.3,-10.6,8.3,0.0,0.7


#### Loading in the Fabry-Perot data

In [8]:
from datetime import datetime, timedelta

def adjust_date(row):
    '''This function adds one day to the date if above 24, as the date from each .dat file is over two days but only written down as one

    Inputs:
        row: the row of the dataframe for the date to be adjusted

    Returns:
        row: the row with the adjusted datetime
    '''
    new_time = float(row['time'])

    #have to set as a float becuase when used this 
    if float(row['time']) > 24:
        row['date'] += timedelta(days=1)
        new_time = float(row['time']) - 24 

    # Convert decimal hours to seconds
    seconds = round(new_time * 3600)

    # Create a timedelta object
    time_delta = timedelta(seconds=seconds)
    
    # Use a starting date to add the timedelta and extract the time
    start_date = datetime(1900, 1, 1)
    result_time = (start_date + time_delta).time()
    row['time'] = str(result_time)

    # need to create a unified datetime object with the date and time objects 
    time_object = datetime.strptime(str(result_time), "%H:%M:%S")
    datetime_combined = datetime(row['date'].year, row['date'].month, row['date'].day,
                             time_object.hour, time_object.minute, time_object.second)
    
    # assign the datetime value to the entry in the date column
    row['date'] = datetime_combined 
    return row

def fpi_file_loader(directory):
    '''This function loads in all of the .dat files and concatenate them simultaneously, performing some data trabformations along the way for ease of use later on

    Inputs:
        directory: the directory of fpi .dat files
        
    Returns:
        dataframe: a concatenated pandas dataframe of all of the .dat files containing the fpi data
    '''
    files = os.listdir(directory)

    # Filter only CSV files
    dat_files = [file for file in files if file.endswith('.dat')]

    # Initialize an empty DataFrame to store the concatenated data
    dataframe = pd.DataFrame()

    # Iterate over each CSV file and append it to the end of the concatenated DataFrame
    for file in dat_files:
        filepath = os.path.join(directory, file)

        #extract date string from .dat filename
        date_string = '20' + str(file[1:3]) + '-' + str(int(file[3],16)) + '-' + str(file[4:6])

        # Define the format of the date string
        date_format = "%Y-%m-%d"

        # Convert the string to a datetime object
        date_object = datetime.strptime(date_string, date_format)

        # Read lines from the file starting from line 14 as the data above isn't tabular it is in written form so unusable if unspecified (check a FP .dat file to see what I mean)
        with open(filepath, 'r') as file:
            lines = file.readlines()[13:]

        # Create a DataFrame from the lines and add the date column
        df = pd.DataFrame([line.strip().split() for line in lines])
        df['date'] = date_object
        df.rename(columns={0: "time", 1: "mirror", 2: "intensity", 3: "intensityError", 4:"windSpeed", 5: "windSpeedError", 6: "temp", 7:"tempError", 8:"chiSquared", 9:"sigToNoiseRatio", 10:"peak"}, inplace=True)

        # Apply the function based on the condition of if more than 24 , change the date to the net day
        df = df.apply(adjust_date, axis=1)
        df.rename(columns={'date': 'datetime'}, inplace=True)

        #concatenate new df to master dataframe
        dataframe = pd.concat([dataframe, df])
    
    # setting the correct datatypes and dropping 'peak' column as it is irrelevant
    dataframe = dataframe.drop(['peak', 'time'], axis=1)
    
    dataframe['mirror'] = dataframe['mirror'].astype(int)
    dataframe['intensity'] = dataframe['intensity'].astype(float)
    dataframe['intensityError'] = dataframe['intensityError'].astype(float)
    dataframe['windSpeed'] = dataframe['windSpeed'].astype(float)
    dataframe['windSpeedError'] = dataframe['windSpeedError'].astype(float)
    dataframe['temp'] = dataframe['temp'].astype(float)
    dataframe['tempError'] = dataframe['tempError'].astype(float)
    dataframe['chiSquared'] = dataframe['chiSquared'].astype(float)
    dataframe['sigToNoiseRatio'] = dataframe['sigToNoiseRatio'].astype(float)

    # sort the dataframe by date descedning and reset the idnex for a clean dataset
    dataframe = dataframe.sort_values(by='datetime')
    dataframe = dataframe.reset_index(drop=True)
    
    return dataframe

In [9]:
sodankyla_red_fpi_data = fpi_file_loader(sodankyla_dir)
sodankyla_red_fpi_data.to_csv(sodankyla_dir + 'sodankyla_red_fpi_data.csv', index=False)
sodankyla_red_fpi_data

,mirror,intensity,intensityError,windSpeed,windSpeedError,temp,tempError,chiSquared,sigToNoiseRatio,datetime
0,8,266.13,29.46,438.81,82.61,536.20,174.33,0.12,58.0,2003-02-03 01:32:32
1,8,179.75,17.91,-834.91,90.01,1495.74,414.75,0.12,64.0,2003-02-03 03:02:19
2,7,805.21,99.31,227.00,73.92,500.46,153.06,0.09,42.0,2003-02-03 03:16:02
3,3,876.37,120.74,299.93,76.72,401.64,135.78,0.09,33.0,2003-02-03 03:17:18
4,1,164.10,20.90,895.31,78.07,582.49,185.62,0.08,31.0,2003-02-03 05:15:44
...,...,...,...,...,...,...,...,...,...,...
73092,4,11350.52,140.89,2.41,9.34,2133.92,77.00,0.12,6100.0,2023-02-14 05:07:32
73093,14,12052.06,142.05,25.17,8.96,2094.00,71.97,0.14,6595.0,2023-02-14 05:08:40
73094,1,13253.51,150.48,37.98,8.38,2200.38,72.17,0.19,7616.0,2023-02-14 05:09:47
73095,8,13009.21,72.20,-124.14,4.54,1606.46,26.12,0.58,23378.0,2023-02-14 05:10:16


In [10]:
kiruna_red_fpi_data = fpi_file_loader(kiruna_dir + 'Kiruna_Red/')
kiruna_red_fpi_data.to_csv(kiruna_dir + 'kiruna_red_fpi_data.csv', index=False)
kiruna_red_fpi_data

,mirror,intensity,intensityError,windSpeed,windSpeedError,temp,tempError,chiSquared,sigToNoiseRatio,datetime
0,1,10061.11,30.36,-176.30,1.76,322.93,2.51,0.11,64516.0,2003-02-01 00:00:30
1,11,7920.98,26.61,-129.59,3.00,1317.83,11.44,0.13,52114.0,2003-02-01 00:01:01
2,2,6211.83,20.62,-12.20,2.94,1288.63,11.03,0.15,53247.0,2003-02-01 00:01:30
3,8,2557.50,37.31,-57.17,7.79,437.18,15.46,2.92,4510.0,2003-02-01 00:02:01
4,12,3327.00,17.80,87.44,4.84,1351.17,18.74,0.10,20568.0,2003-02-01 00:02:30
...,...,...,...,...,...,...,...,...,...,...
401244,11,2359.47,15.68,36.63,6.17,1506.20,26.86,0.11,13974.0,2023-03-01 04:26:09
401245,2,2134.76,9.79,42.63,4.29,1534.23,18.94,0.24,29493.0,2023-03-01 04:26:45
401246,8,10003.92,20.56,3.70,1.43,768.21,3.96,0.96,133738.0,2023-03-01 04:27:24
401247,12,1737.01,10.82,49.97,5.80,1523.40,25.51,0.11,15919.0,2023-03-01 04:28:02


#### Merging the two datasets 

Going to have to find an approximate method to match up the times in each dataset, as they are not always mergable. Will find the cloud data which is closest in time to each entry in the FPI dataset, and then merge these values for each row to make a new dataset.

We're going to say that if the datetime in the FPI data is within an hour of any data in the cloud data, then we can use it, if not then we can discard it (this can be changed depednant on if it is accurate engough ie. smaller timescale)

In [11]:
import pandas as pd
from tqdm import tqdm

def find_closest_datetime_with_values(fpi_df, cloud_df, hours=0.1):
    '''This function finds the row in the cloud_df, which has the closesst datetime value to the row in the fpi_df, anbd merges this row to the fpi_df

    Inputs:
       fpi_df: the fabry perot dataframe (created from the functions above)
       cloud_df: the cloud_sensor dataframe (created from the functions above)
       hours: the time difference in hours to be used to find the nearest datetime value(default 0.1)

    Returns:
        dataframe: a pandas dataframe of the merged fpi and cloud data
    '''
    # Ensure 'datetime' columns are of datetime type
    fpi_df['datetime'] = pd.to_datetime(fpi_df['datetime'])
    cloud_df['datetime'] = pd.to_datetime(cloud_df['datetime'])
    
    # Sort the second dataframe by 'datetime' for efficient searching
    cloud_df = cloud_df.sort_values(by='datetime').reset_index(drop=True)

    # the columns which are going to be merged onto the fpi_df from the cloud_df
    additional_columns = cloud_df.keys()[1:].to_list()
    
    def get_closest(row):
        '''This function finds a row in the cloud_df which is closest in datetime to a row in the fpi_df, to be used to megre the two dataframes in the parent function

        Inputs:
            row: the row in the fpi_df to match with the cloud_df

        Returns:
            closest_row: the row in cloud_df with the closest datatime value to the row in the fpi_df
        '''
        dt = row['datetime']
        # Find the index of the closest datetime in cloud_df
        time_diffs = (cloud_df['datetime'] - dt).abs()
        min_time_diff = time_diffs.min()
        if min_time_diff > pd.Timedelta(hours=hours):
            # If the closest time difference is more <hours>, return None values
            return pd.Series([pd.NaT] + [None]*len(additional_columns), index=['datetime'] + additional_columns)
        idx = time_diffs.idxmin()
        # Retrieve the closest datetime and additional columns
        closest_row = cloud_df.loc[idx,  additional_columns]
        return closest_row
    
    # Initialize a DataFrame to store results
    result_df = pd.DataFrame(columns= additional_columns, index=fpi_df.index)
    
    # Iterate through each row in fpi_df with tqdm
    for index, row in tqdm(fpi_df.iterrows(), total=fpi_df.shape[0], desc="Processing rows"):
        closest_row = get_closest(row)
        result_df.loc[index] = closest_row
    
    # Concatenate the original fpi_df with the closest rows
    result_df = pd.concat([fpi_df.reset_index(drop=True), result_df.reset_index(drop=True)], axis=1)
    dropped = result_df.dropna()

    #reset the index
    dataframe = dropped.reset_index(drop=True)
    
    return dataframe

In [13]:
# specify the closeness of the datetime values to be merged in hours (change this accordingly, if the <hours> paramter is too small, it will return a small dataframe
# as it returns None values and drops them if there are no values within that specified time difference)
hours = 0.1
# Find closest date time with values
sodankyla_red_merged = find_closest_datetime_with_values(sodankyla_red_fpi_data, sodankyla_cloud_data, hours=hours)

Processing rows: 100%|██████████| 73097/73097 [04:47<00:00, 253.90it/s]


In [14]:
sodankyla_red_merged.to_csv(sodankyla_dir + f'sodankyla_red_merged({hours}_hours_diff).csv', index=False)
sodankyla_red_merged

,mirror,intensity,intensityError,windSpeed,windSpeedError,temp,tempError,chiSquared,sigToNoiseRatio,datetime,sensorTemp,skyTemp,clarity,light,rain
0,15,19445.41,105.44,-21.77,3.26,409.98,6.53,0.12,24868.0,2019-02-01 15:16:03,2.3,-40.6,42.8,0.0,0.0
1,4,25100.11,79.40,-27.46,2.57,1567.10,14.51,0.37,71291.0,2019-02-01 15:19:09,2.3,-40.8,43.0,0.0,-0.2
2,14,31737.28,106.74,-7.93,2.73,1540.68,15.14,0.38,62523.0,2019-02-01 15:19:46,2.3,-41.7,44.0,0.0,-0.2
3,1,29767.50,98.48,-19.43,2.69,1547.51,14.96,0.37,64765.0,2019-02-01 15:22:52,2.2,-40.4,42.6,0.0,-0.1
4,8,19014.86,69.20,-53.74,2.95,1611.52,17.21,0.59,54783.0,2019-02-01 15:23:21,2.2,-40.4,42.6,0.0,-0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61860,4,11350.52,140.89,2.41,9.34,2133.92,77.00,0.12,6100.0,2023-02-14 05:07:32,10.8,-7.2,18.0,0.0,-0.1
61861,14,12052.06,142.05,25.17,8.96,2094.00,71.97,0.14,6595.0,2023-02-14 05:08:40,10.8,-7.1,18.0,0.0,-0.1
61862,1,13253.51,150.48,37.98,8.38,2200.38,72.17,0.19,7616.0,2023-02-14 05:09:47,10.8,-6.7,17.5,0.0,-0.2
61863,8,13009.21,72.20,-124.14,4.54,1606.46,26.12,0.58,23378.0,2023-02-14 05:10:16,10.8,-6.7,17.5,0.0,-0.2


In [15]:
kiruna_red_merged = find_closest_datetime_with_values(kiruna_red_fpi_data, kiruna_cloud_data, hours)

Processing rows: 100%|██████████| 401249/401249 [14:40<00:00, 455.56it/s]


In [16]:
kiruna_red_merged.to_csv(kiruna_dir +  f'kiruna_red_merged({hours}_hours_diff).csv', index=False)
kiruna_red_merged

,mirror,intensity,intensityError,windSpeed,windSpeedError,temp,tempError,chiSquared,sigToNoiseRatio,datetime,sensorTemp,skyTemp,clarity,light,rain
0,8,1036.74,2.41,14.41,1.09,215.49,1.27,2.36,111212.0,2019-02-01 15:04:37,-2.5,-8.3,5.8,0.0,0.6
1,8,3315.85,4.06,1901.73,0.49,242.92,0.71,8.44,275712.0,2019-02-01 15:04:46,-2.5,-8.3,5.8,0.0,0.6
2,8,1668.01,2.95,2.65,1.31,882.96,3.96,2.57,190631.0,2019-02-01 15:10:17,-2.6,-8.4,5.9,0.0,0.7
3,8,3305.77,3.49,1905.15,0.43,244.44,0.62,11.96,372759.0,2019-02-01 15:10:28,-2.6,-8.4,5.9,0.0,0.7
4,8,1674.81,3.05,12.87,1.35,886.85,4.10,2.25,179485.0,2019-02-01 15:16:08,-2.5,-8.2,5.7,0.0,0.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61424,7,2114.59,10.77,88.60,4.64,1430.11,19.41,0.13,23592.0,2023-03-01 00:02:19,8.2,-4.5,12.6,0.0,0.0
61425,2,2929.48,12.49,138.31,3.82,1377.23,15.57,0.12,33577.0,2023-03-01 00:03:04,8.2,-4.5,12.6,0.0,0.0
61426,8,9813.38,17.64,13.98,1.26,777.89,3.51,1.30,175454.0,2023-03-01 00:03:43,8.2,-4.5,12.6,0.0,0.0
61427,12,2534.86,9.84,141.08,3.54,1440.30,14.91,0.13,40700.0,2023-03-01 00:04:20,8.2,-4.5,12.6,0.0,0.0
